In [1]:
!pip install huggingface_hub jiwer evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 39.0 MB/s eta 0:00:00


In [2]:
!hf auth login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? [y/N]: y
Token is valid (permission: fineGrained).
The token `token` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate w

In [3]:
from datasets import load_dataset, Audio
from transformers import pipeline
import evaluate
import torch
import re

In [31]:
dataset = load_dataset('KoelLabs/L2Arctic')
dataset = dataset['scripted']

print(dataset)

Dataset({
    features: ['audio', 'ipa', 'text', 'speaker_code', 'speaker_gender', 'speaker_native_language'],
    num_rows: 3599
})


In [32]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))

In [33]:
dataset = dataset.map(lambda x: {'ipa': x['ipa'].replace("ˈ", "")})

In [34]:
dataset[0]

{'audio': <datasets.features._torchcodec.AudioDecoder at 0x7ff7b407af00>,
 'ipa': 'fɔɹðʌtwɛntiɪθtaɪmðætivnɪŋkðʌtumɛnʃʊkhænss',
 'text': 'for the twentieth time that evening the two men shook hands',
 'speaker_code': 'ABA',
 'speaker_gender': 'm',
 'speaker_native_language': 'Arabic'}

In [35]:
device_id = 0 if torch.cuda.is_available() else -1
device_name = "GPU" if device_id == 0 else "CPU"

print(f"Using: {device_name}")

model_id = 'marx90/lingulu_wav2vec2_pronounciation_finetune'

Using: GPU


In [36]:
cer_metric = evaluate.load("cer")

In [37]:
pipe = pipeline("automatic-speech-recognition", model=model_id, device=device_id)

Loading weights:   0%|          | 0/424 [00:00<?, ?it/s]

In [27]:
def evaluate_sample(batch):
    prediction = pipe(batch["audio"]["array"])['text']

    batch["prediction"] = prediction.lower()
    batch["reference"] = batch["ipa"].lower()
    return batch

In [38]:
results = dataset.map(evaluate_sample)

Map:   0%|          | 0/3599 [00:00<?, ? examples/s]

In [39]:
predictions = results["prediction"]
references = results["reference"]

In [40]:
score = cer_metric.compute(predictions=predictions, references=references)

print(f"CER score: {score}")

CER score: 0.20780067935755367
